In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
import spotipy
import spotipy.oauth2 as oauth2

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials

client_id='2264c5e5d4c1469d812c8097babe7e42'
client_secret='2050b324730b46aea17caa8e51175da9'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [26]:
import os

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return [ filename for filename in filenames if (filename.endswith(suffix) and filename.startswith('all_playlist'))]

In [57]:
csvs = find_csv_filenames(os.getcwd())
csvs.sort(reverse=True)

In [58]:
csvs

['all_playlist_ids2019-08-23.csv',
 'all_playlist_ids2019-08-07.csv',
 'all_playlist_ids2019-07-05.csv',
 'all_playlist_ids2019-06-22.csv',
 'all_playlist_ids2019-05-23.csv',
 'all_playlist_ids2019-04-29.csv',
 'all_playlist_ids2019-04-23.csv',
 'all_playlist_ids2019-04-22.csv',
 'all_playlist_ids2019-04-19.csv',
 'all_playlist_ids2019-04-08.csv',
 'all_playlist_ids2019-04-06.csv',
 'all_playlist_ids2019-04-05.csv',
 'all_playlist_ids2019-01-09.csv',
 'all_playlist_ids2018-12-25.csv',
 'all_playlist_ids2018-11-22.csv',
 'all_playlist_ids2018-11-15.csv',
 'all_playlist_ids2018-10-31.csv',
 'all_playlist_ids2018-10-19.csv',
 'all_playlist_ids2018-10-12.csv',
 'all_playlist_ids2018-10-10.csv',
 'all_playlist_ids2018-09-13.csv',
 'all_playlist_ids2018-09-12.csv',
 'all_playlist_ids2018-09-11.csv']

In [59]:
csvs = csvs[2:].copy()

In [ ]:
dfs = [pd.read_csv(csv, headers=False) for df in csvs]

In [2]:
def update_word_counts(playlist_id):
    try:
        result = spot.user_playlist(user = None, playlist_id= playlist_id)
        playlist_name = result['name']
        tracks = result['tracks']['items']
        tracklist = []
        for track in tracks:
            try:
                tid = track['track']['id']
                tracklist.append(tid)
            except:
                continue
        df = pd.DataFrame(tracklist)
        df['playlist_id'] = playlist_id
        df['playlist_name'] = result['name']
        df['user_id'] = result['owner']['id']
        try:
            df['followers'] = result['followers']['total']
        except:
            np.nan
        return (playlist_id,df)
    
    except spotipy.SpotifyException:
        return (0, playlist_id)


In [5]:
result = spot.user_playlist(playlist_id='0s5y6uoZh4RAHnOuPEqhlg?si=V2PD78muSRabd_Jggh3FiA',user = None)

In [7]:
result['owner']['id']

'mfingerhood'

In [ ]:
i=1
for df in dfs:
    print(f'start {i}')
    df.dropna(inplace=True)
    df.drop_duplicates(subset=['playlist_ids'],inplace=True)
    df.reset_index(inplace=True)
    
    framelist = []
#     fails = []

    for i in tqdm(range(len(df.playlist_ids))):
        x = update_word_counts(df.playlist_ids[i])
        if x[0] ==0:
            fails.append(x[1])
        else:
            framelist.append(x[1])
    
    df = pd.concat(framelist)
    df.columns = ['song_id', 'playlist_id', 'playlist_name', 'user_id', 'followers']
    df.to_csv('new_scraped_playlists.csv', mode='a', header=False)
    i+=1

In [21]:
df = pd.concat(framelist)

In [23]:
df.columns = ['song_id', 'playlist_id', 'playlist_name', 'user_id', 'followers']

In [25]:
df.to_csv('new_scraped_playlists.csv', mode='a', header=False)